In [1]:
from google.colab import drive
import librosa
import numpy as np
import soundfile as sf
import os
import pandas as pd
import re

# Mount Google Drive
drive.mount('/content/drive')

# Define base directory in Google Drive
base_dir = '/content/drive/MyDrive/Data Science AUEB/Deep Learning /datasets/'

# Update the file path to reflect the Google Drive directory structure
file_path = os.path.join(base_dir, 'labeled wavs', 'Initial_wav_labeled.csv')
data = pd.read_csv(file_path)

# Define the augmentation functions with specific parameters
def add_noise(wav, noise_factor=0.005):
    """Add white noise to the audio signal with a fixed noise factor."""
    noise = np.random.randn(len(wav))  # Generate white noise
    return wav + noise_factor * noise  # Add the noise to the original signal

def pitch_shift(wav, sr, pitch_factors=[-2, 1]):
    """Pitch shift the audio signal by the specified pitch shift factors."""
    augmented_audio = []
    for pitch_factor in pitch_factors:
        shifted_audio = librosa.effects.pitch_shift(wav, sr=sr, n_steps=pitch_factor)
        augmented_audio.append(shifted_audio)
    return augmented_audio  # Return a list of pitch-shifted versions

def time_stretch(wav, stretch_factors=[0.4, 0.17]):
    """Time stretch the audio signal by the specified stretch factors."""
    augmented_audio = []
    for stretch_factor in stretch_factors:
        stretched_audio = librosa.effects.time_stretch(wav, rate=stretch_factor)
        augmented_audio.append(stretched_audio)
    return augmented_audio  # Return a list of time-stretched versions

# Function to apply augmentations
def augment_audio(audio, sr):
    """
    Apply the specified augmentations (time stretching, pitch shifting, noise addition)
    on the audio signal.

    Returns:
    - A list of augmented versions for each type of transformation.
    """
    # Apply time stretching with factors [0.4, 0.17]
    time_stretched = time_stretch(audio, stretch_factors=[0.4, 0.17])

    # Apply pitch shifting with factors [-2, 1]
    pitch_shifted = pitch_shift(audio, sr, pitch_factors=[-2, 1])

    # Add noise with a factor of 0.005
    noisy_audio = add_noise(audio, noise_factor=0.005)

    # Return a list of all augmented versions
    return time_stretched + pitch_shifted + [noisy_audio]

def save_augmented_files(directory, save_directory, minority_classes, data, augment=False):
    """Load audio files, apply augmentations, and save the augmented files."""
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    files_processed = 0
    total_files_saved = 0

    # Get the list of filenames from the CSV for the minority classes
    filenames_in_csv = data[data['Diagnosis'].isin(minority_classes)]['Filename'].str.strip().tolist()
    print(f"Total filenames in CSV for minority classes: {len(filenames_in_csv)}")

    # Get all .wav files in the specified directory
    all_wav_files = [f for f in os.listdir(directory) if f.endswith(".wav")]
    print(f"Total .wav files in the directory: {len(all_wav_files)}")

    # Create a set of base filenames from the CSV for faster lookup
    base_filenames_in_csv = {re.sub(r'_snippet_\d+', '', fname.split('.wav')[0]) for fname in filenames_in_csv}
    print(f"Total unique base filenames in CSV: {len(base_filenames_in_csv)}")

    for filename in all_wav_files:
        base_filename_current = re.sub(r'_snippet_\d+', '', filename.split('.wav')[0])

        if base_filename_current in base_filenames_in_csv:
            files_processed += 1
            file_path = os.path.join(directory, filename)

            try:
                audio, sr = librosa.load(file_path, sr=None)
                print(f"Audio loaded: {file_path} | Audio shape: {audio.shape}, Sample rate: {sr}")
            except Exception as e:
                print(f"Error loading audio {file_path}: {e}")
                continue

            augmented_audio_list = augment_audio(audio, sr)

            if not augmented_audio_list:
                print(f"No augmented audio generated for {filename}")
            else:
                print(f"Generated {len(augmented_audio_list)} augmented versions for {filename}")

            for i, augmented_audio in enumerate(augmented_audio_list):
                transformation_type = ["stretch_0.4", "stretch_0.17", "pitch_-2", "pitch_1", "noise_0.005"][i]
                augmented_filename = f"{filename}_{transformation_type}.wav"
                augmented_filepath = os.path.join(save_directory, augmented_filename)

                print(f"Saving {augmented_filename} to {augmented_filepath}")

                try:
                    sf.write(augmented_filepath, augmented_audio, sr)
                    total_files_saved += 1
                    print(f"Saved {augmented_filename}")
                except Exception as e:
                    print(f"Failed to save {augmented_filename}: {e}")

    print(f"Processed {files_processed} files.")
    print(f"Total augmented files saved: {total_files_saved}")


# Example directory and minority classes (adjust paths for Google Drive)
directory = '/content/drive/MyDrive/Data Science AUEB/Deep Learning /datasets/preprocessed_initial_wav'  # Adjust this path
minority_classes = ['Bronchiectasis']
save_directory = '/content/drive/MyDrive/Data Science AUEB/Deep Learning /datasets/augmented_wav_files'  # Adjust this path

# Apply augmentations and save the files
save_augmented_files(directory, save_directory, minority_classes, data)

Mounted at /content/drive
Total filenames in CSV for minority classes: 25
Total .wav files in the directory: 11562
Total unique base filenames in CSV: 25
Audio loaded: /content/drive/MyDrive/Data Science AUEB/Deep Learning /datasets/preprocessed_initial_wav/196_1b1_Pr_sc_Meditron_snippet_1.wav | Audio shape: (220500,), Sample rate: 44100
Generated 5 augmented versions for 196_1b1_Pr_sc_Meditron_snippet_1.wav
Saving 196_1b1_Pr_sc_Meditron_snippet_1.wav_stretch_0.4.wav to /content/drive/MyDrive/Data Science AUEB/Deep Learning /datasets/augmented_wav_files/196_1b1_Pr_sc_Meditron_snippet_1.wav_stretch_0.4.wav
Saved 196_1b1_Pr_sc_Meditron_snippet_1.wav_stretch_0.4.wav
Saving 196_1b1_Pr_sc_Meditron_snippet_1.wav_stretch_0.17.wav to /content/drive/MyDrive/Data Science AUEB/Deep Learning /datasets/augmented_wav_files/196_1b1_Pr_sc_Meditron_snippet_1.wav_stretch_0.17.wav
Saved 196_1b1_Pr_sc_Meditron_snippet_1.wav_stretch_0.17.wav
Saving 196_1b1_Pr_sc_Meditron_snippet_1.wav_pitch_-2.wav to /cont